In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import time
import os
import sys
import argparse
import matplotlib.pyplot as plt
from torch_geometric.data import Data


In [2]:


from dotenv import dotenv_values

SECRETS = dotenv_values("../envs/graphs.env")

TRAIN_DATA_PATH = SECRETS['TRAIN_DATA_PATH']
TEST_DATA_PATH = SECRETS['TEST_DATA_PATH']
PULL_UP_STRENGTH, PULL_DOWN_STRENGTH = list(map(int, SECRETS['BETA'].split(':')))
NUM_SAMPLES = int(SECRETS['NUM_SAMPLES'])
MAX_LENGTH = int(SECRETS['MAX_LENGTH'])

In [5]:
def SIP_Amount(N:int, r:float, a:float, k:int):
    val = np.sum([( min(k*(1+a*(i//12)), 25000000) ) *((1+r/12)**(N*12-i)) for i in range (N*12+1)])    
    invested = np.sum([min(k*(1+a*(i//12)), 25000000) for i in range (N*12+1)])
    # val = np.sum([(k*(1+a*(i//12)) ) *((1+r/12)**(N*12-i)) for i in range (N*12+1)])
    # invested = np.sum([k*(1+a*(i//12)) for i in range (N*12+1)])
    return f"{val:,}", f"{invested:,}"

SIP_Amount(N=20, r=0.081, a=0.05, k=30000)

('24,310,265.924565215', '10,680,000.0')

In [4]:
data = np.load(TRAIN_DATA_PATH+f"/train_data/circuit{1}.npz")

In [5]:
train_data = [ (np.load(TRAIN_DATA_PATH + f"/train_data/circuit{i}.npz")['adj_list'],  np.load(TRAIN_DATA_PATH + f"/train_data/circuit{i}.npz")['feature_matrix']) for i in range(1, 1000+1) ]